<a href="https://colab.research.google.com/github/feliciahf/NLP-Project/blob/main/amazon/colab_FH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

Check whether stemming & lemmatization make difference

Do we need bigram/trigram frequencies (within titles)?

Do we need dataframe including cleaned data & genres? (which cleaned data?) -> create dictionary of lists maybe
https://www.geeksforgeeks.org/python-ways-to-create-a-dictionary-of-lists/

How do we incorporate POS-tagging & NER in classifier?

# Importing & Cleaning data

This section imports the data into a pandas dataframe and goes through the following preprocessing steps:

-Case collapsing 
-Remove punctuation 
-Tokenization 
-N-Grams: bigrams and trigrams 
-Stemming -> check whether this makes a difference 
-Lemmatization -> check whether this makes a difference 
-Part-of-speech (POS) tagging 
-Named entity recognition (NER) 

## Import Data
Imports data as pandas dataframe
Create list of title and genre columns from original data
Create list including all 32 genres

In [3]:
from google.colab import files
uploaded = files.upload()

Saving book32listing.csv to book32listing.csv


In [4]:
import pandas as pd
df = pd.read_csv("book32listing.csv", encoding='latin1', header=None)
df1 = df[[3,6]] # only columns with titles and genres
df1.columns = ['title', 'genre']
print(df1)

                                                    title      genre
0                         Mom's Family Wall Calendar 2016  Calendars
1                         Doug the Pug 2016 Wall Calendar  Calendars
2       Moleskine 2016 Weekly Notebook, 12M, Large, Bl...  Calendars
3                 365 Cats Color Page-A-Day Calendar 2016  Calendars
4                    Sierra Club Engagement Calendar 2016  Calendars
...                                                   ...        ...
207567  ADC the Map People Washington D.C.: Street Map...     Travel
207568  Washington, D.C., Then and Now: 69 Sites Photo...     Travel
207569  The Unofficial Guide to Washington, D.C. (Unof...     Travel
207570      Washington, D.C. For Dummies (Dummies Travel)     Travel
207571  Fodor's Where to Weekend Around Boston, 1st Ed...     Travel

[207572 rows x 2 columns]


In [5]:
titles = df1['title'] # list of all titles
titles1 = titles.values.tolist() # change to list of strings
print(titles1[0:6]) # test whether it worked

["Mom's Family Wall Calendar 2016", 'Doug the Pug 2016 Wall Calendar', 'Moleskine 2016 Weekly Notebook, 12M, Large, Black, Soft Cover (5 x 8.25)', '365 Cats Color Page-A-Day Calendar 2016', 'Sierra Club Engagement Calendar 2016', 'Sierra Club Wilderness Calendar 2016']


In [6]:
genres = df1['genre']
genres = genres.values.tolist()
genres = pd.DataFrame(genres)

In [7]:
df1.genre.unique() # list of all possible genres

array(['Calendars', 'Comics & Graphic Novels', 'Test Preparation',
       'Mystery, Thriller & Suspense', 'Science Fiction & Fantasy',
       'Romance', 'Humor & Entertainment', 'Literature & Fiction',
       'Gay & Lesbian', 'Engineering & Transportation',
       'Cookbooks, Food & Wine', 'Crafts, Hobbies & Home',
       'Arts & Photography', 'Education & Teaching',
       'Parenting & Relationships', 'Self-Help', 'Computers & Technology',
       'Medical Books', 'Science & Math', 'Health, Fitness & Dieting',
       'Business & Money', 'Law', 'Biographies & Memoirs', 'History',
       'Politics & Social Sciences', 'Reference',
       'Christian Books & Bibles', 'Religion & Spirituality',
       'Sports & Outdoors', 'Teen & Young Adult', "Children's Books",
       'Travel'], dtype=object)

## Case Collapsing
Change all uppercase to lowercase letters

In [8]:
case_collap = map(lambda x:x.lower(), titles1)
case_collap_list = list(case_collap)
print(case_collap_list[0:6])

["mom's family wall calendar 2016", 'doug the pug 2016 wall calendar', 'moleskine 2016 weekly notebook, 12m, large, black, soft cover (5 x 8.25)', '365 cats color page-a-day calendar 2016', 'sierra club engagement calendar 2016', 'sierra club wilderness calendar 2016']


## Remove Punctuation
Remove punctuation by creating translation table
Punctuation to be removed is given in string: string.punctuation

In [9]:
import string
trans = str.maketrans('', '', string.punctuation)
rem_punct = [s.translate(trans) for s in case_collap_list]
print(rem_punct[0:6])

['moms family wall calendar 2016', 'doug the pug 2016 wall calendar', 'moleskine 2016 weekly notebook 12m large black soft cover 5 x 825', '365 cats color pageaday calendar 2016', 'sierra club engagement calendar 2016', 'sierra club wilderness calendar 2016']


## Tokenization
Split all titles into words
Output: list of lists of strings

In [10]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
tokenized_titles = [word_tokenize(i) for i in rem_punct]
print(tokenized_titles[:10])

[['moms', 'family', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskine', '2016', 'weekly', 'notebook', '12m', 'large', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cats', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engagement', 'calendar', '2016'], ['sierra', 'club', 'wilderness', 'calendar', '2016'], ['thomas', 'kinkade', 'the', 'disney', 'dreams', 'collection', '2016', 'wall', 'calendar'], ['ansel', 'adams', '2016', 'wall', 'calendar'], ['dilbert', '2016', 'daytoday', 'calendar'], ['mary', 'engelbreit', '2016', 'deluxe', 'wall', 'calendar', 'never', 'give', 'up']]


## N-Grams: Bigrams and Trigrams
Create bigrams + trigrams
(could be done in one go e.g. n=2 for bigrams, n=3 for trigrams etc.

In [12]:
# bigrams
token_bigram = []
for title in tokenized_titles:
    title_bigram = []
    for w in range(len(title) - 1):
        title_bigram.append([title[w], title[w + 1]])
    token_bigram.append(title_bigram)
print(token_bigram[:6]) # test whether working

[[['moms', 'family'], ['family', 'wall'], ['wall', 'calendar'], ['calendar', '2016']], [['doug', 'the'], ['the', 'pug'], ['pug', '2016'], ['2016', 'wall'], ['wall', 'calendar']], [['moleskine', '2016'], ['2016', 'weekly'], ['weekly', 'notebook'], ['notebook', '12m'], ['12m', 'large'], ['large', 'black'], ['black', 'soft'], ['soft', 'cover'], ['cover', '5'], ['5', 'x'], ['x', '825']], [['365', 'cats'], ['cats', 'color'], ['color', 'pageaday'], ['pageaday', 'calendar'], ['calendar', '2016']], [['sierra', 'club'], ['club', 'engagement'], ['engagement', 'calendar'], ['calendar', '2016']], [['sierra', 'club'], ['club', 'wilderness'], ['wilderness', 'calendar'], ['calendar', '2016']]]


In [13]:
# trigrams
token_trigram = []
for title in tokenized_titles:
    title_trigram = []
    for w in range(len(title) - 2):
        title_trigram.append([title[w], title[w + 1], title[w + 2]])
    token_trigram.append(title_trigram)
print(token_trigram[:6]) # test whether working

[[['moms', 'family', 'wall'], ['family', 'wall', 'calendar'], ['wall', 'calendar', '2016']], [['doug', 'the', 'pug'], ['the', 'pug', '2016'], ['pug', '2016', 'wall'], ['2016', 'wall', 'calendar']], [['moleskine', '2016', 'weekly'], ['2016', 'weekly', 'notebook'], ['weekly', 'notebook', '12m'], ['notebook', '12m', 'large'], ['12m', 'large', 'black'], ['large', 'black', 'soft'], ['black', 'soft', 'cover'], ['soft', 'cover', '5'], ['cover', '5', 'x'], ['5', 'x', '825']], [['365', 'cats', 'color'], ['cats', 'color', 'pageaday'], ['color', 'pageaday', 'calendar'], ['pageaday', 'calendar', '2016']], [['sierra', 'club', 'engagement'], ['club', 'engagement', 'calendar'], ['engagement', 'calendar', '2016']], [['sierra', 'club', 'wilderness'], ['club', 'wilderness', 'calendar'], ['wilderness', 'calendar', '2016']]]


## Stemming
Test this out to see whether it makes a difference in final classifier

PorterStemmer (one algorithm for stemming; less aggressive than LancasterStemming)

Create empty list to contain lists of stems in each title
Create empty list for stems of title
Add each stemmed word in title to second list
Append list of stemmed words to first list

In [14]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()
stems = []   
for title in tokenized_titles:
    stems_title = []
    for word in title:
        stems_title.append(porter.stem(word))
    stems.append(stems_title)
    
print(stems[0:6]) # test whether it works

[['mom', 'famili', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskin', '2016', 'weekli', 'notebook', '12m', 'larg', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cat', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engag', 'calendar', '2016'], ['sierra', 'club', 'wilder', 'calendar', '2016']]


## Lemmatization
Same principle as with stemming
Test this out to see whether it makes a difference in final classifier

In [15]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
lemmas = []
for title in tokenized_titles:
    lemmas_title = []
    for word in title:
        lemmas_title.append(lemmatizer.lemmatize(word))
    lemmas.append(lemmas_title)
    
print(lemmas[0:6]) # test whether it works

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[['mom', 'family', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskine', '2016', 'weekly', 'notebook', '12m', 'large', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cat', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engagement', 'calendar', '2016'], ['sierra', 'club', 'wilderness', 'calendar', '2016']]


## Part-of-Speech (POS) Tagging
Create list of lists with tokens and their corresponding part-of-speech tag in each title

In [16]:
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag

postag = []
for title in tokenized_titles:
    postag.append(nltk.pos_tag(title))
    
print(postag[0:6]) # testing whether postag worked

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[[('moms', 'NNS'), ('family', 'NN'), ('wall', 'NN'), ('calendar', 'NN'), ('2016', 'CD')], [('doug', 'VB'), ('the', 'DT'), ('pug', 'NN'), ('2016', 'CD'), ('wall', 'NN'), ('calendar', 'NN')], [('moleskine', 'NN'), ('2016', 'CD'), ('weekly', 'JJ'), ('notebook', 'NN'), ('12m', 'CD'), ('large', 'JJ'), ('black', 'JJ'), ('soft', 'JJ'), ('cover', 'NN'), ('5', 'CD'), ('x', 'JJ'), ('825', 'CD')], [('365', 'CD'), ('cats', 'NNS'), ('color', 'VBP'), ('pageaday', 'IN'), ('calendar', 'NN'), ('2016', 'CD')], [('sierra', 'NN'), ('club', 'NN'), ('engagement', 'NN'), ('calendar', 'NN'), ('2016', 'CD')], [('sierra', 'NN'), ('club', 'NN'), ('wilderness', 'NN'), ('calendar', 'NN'), ('2016', 'CD')]]


## Named Entity Recognition (NER)
Create list of lists with words, correspondent POS and named entity tags for each title
This uses postags created in previous step

In [17]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import ne_chunk
from nltk.chunk import tree2conlltags

ner = []
for title in titles1:
    ner.append(tree2conlltags(ne_chunk(pos_tag(word_tokenize(title)))))
    
print(ner[0:6]) # test whether NER worked

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[[('Mom', 'NNP', 'B-PERSON'), ("'s", 'POS', 'O'), ('Family', 'NNP', 'B-PERSON'), ('Wall', 'NNP', 'I-PERSON'), ('Calendar', 'NNP', 'I-PERSON'), ('2016', 'CD', 'O')], [('Doug', 'NNP', 'O'), ('the', 'DT', 'O'), ('Pug', 'NNP', 'O'), ('2016', 'CD', 'O'), ('Wall', 'NNP', 'B-FACILITY'), ('Calendar', 'NNP', 'I-FACILITY')], [('Moleskine', 'NN', 'O'), ('2016', 'CD', 'O'), ('Weekly', 'NNP', 'O'), ('Notebook', 'NNP', 'O'), (',', ',', 'O'), ('12M', 'CD', 'O'), (',', ',', 'O'), ('Large', 'NNP', 'B-PERSON'), (',', ',', 'O'), ('Black', 'NNP', 'B-PERSON'), (',', ',', 'O'), ('Soft', 'NNP', 'B-PERSON'), ('Cover', 'NNP', 'I-PERSON'), ('(', '(', 'O'), ('5', 'CD', 'O'), ('x', 'RB', 'O'), ('8.25', 'CD', 'O'), (')', ')', 'O')], [('365', 'CD', 'O'), ('Cats', 'NNPS', '

# Classifiers
Check whether stemming/lemmatization make difference in final classifiers
-> does it improve/worsen classifier?


-Naive Bayes (bag-of-words)

-BERT (language model)

https://towardsdatascience.com/text-classification-with-nlp-tf-idf-vs-word2vec-vs-bert-41ff868d1794

In [18]:
# create dataframe containing tokenized titles and genres
tok_title = pd.DataFrame({0: tokenized_titles})
data_in = [tok_title[0], df1["genre"]]
headers = ["titles", "genres"]

data = pd.concat(data_in, axis=1, keys=headers)
print(data[:5])

                                              titles     genres
0               [moms, family, wall, calendar, 2016]  Calendars
1             [doug, the, pug, 2016, wall, calendar]  Calendars
2  [moleskine, 2016, weekly, notebook, 12m, large...  Calendars
3       [365, cats, color, pageaday, calendar, 2016]  Calendars
4         [sierra, club, engagement, calendar, 2016]  Calendars


In [19]:
# split data into train and test
import numpy as np

test_pct=0.2 # split into 80/20%

# create mask
mask = np.random.choice([0, 1], p=[1 - test_pct, test_pct], size=data.shape[0])

# apply mask
data["mask"] = mask
test = data[data["mask"] == 1]
train = data[data["mask"] == 0]

# removing column
test = test.drop("mask", axis="columns").reset_index()
train = train.drop("mask", axis="columns").reset_index()

# remove original indexing data (otherwise we have double indexing)
test = test.drop("index", axis="columns")
train = train.drop("index", axis="columns")

## Naive Bayes classifier

split into training and test data -> 80% / 20%
use n-grams ?

build vocabulary -> need token frequencies across all titles (create sublist with tokenized_titles)
vocabulary for each genre

In [20]:
from collections import Counter

In [21]:
# token frequencies within each title
unigram_count = []
for title in tokenized_titles:
    uni_title = Counter()
    for i in title:
        uni_title[i] += 1
    unigram_count.append(uni_title)

print(unigram_count[:5]) # test

[Counter({'moms': 1, 'family': 1, 'wall': 1, 'calendar': 1, '2016': 1}), Counter({'doug': 1, 'the': 1, 'pug': 1, '2016': 1, 'wall': 1, 'calendar': 1}), Counter({'moleskine': 1, '2016': 1, 'weekly': 1, 'notebook': 1, '12m': 1, 'large': 1, 'black': 1, 'soft': 1, 'cover': 1, '5': 1, 'x': 1, '825': 1}), Counter({'365': 1, 'cats': 1, 'color': 1, 'pageaday': 1, 'calendar': 1, '2016': 1}), Counter({'sierra': 1, 'club': 1, 'engagement': 1, 'calendar': 1, '2016': 1})]


In [22]:
# token frequencies in total vocab
tok_freq = Counter()
for title in train['titles']:
    for i in title:
        tok_freq[i] += 1
        
print(tok_freq)

Counter({'the': 92401, 'and': 54403, 'of': 50780, 'a': 31347, 'to': 27885, 'for': 21215, 'in': 21129, 'guide': 16003, 'edition': 12507, 'book': 8029, 'with': 7853, 'your': 7325, 'from': 6487, 'series': 6359, 'on': 5927, 'how': 5887, 'an': 5175, 'new': 4866, 'life': 4659, 'world': 4019, 'volume': 3995, 'history': 3965, 'you': 3835, '1': 3318, 'recipes': 3193, 'by': 3160, 'american': 3046, 'travel': 2913, 'law': 2749, 'complete': 2732, '2': 2639, 'art': 2606, 'story': 2600, 'health': 2524, 'my': 2503, 'handbook': 2499, 'books': 2383, 'science': 2295, 'calendar': 2217, 'at': 2186, 'best': 2056, 'novel': 2031, 'practice': 2012, 'i': 1996, 'map': 1935, 'guides': 1911, 'cookbook': 1886, 'that': 1851, 'vol': 1840, 'about': 1810, 'love': 1802, 'what': 1767, '2016': 1761, 'great': 1690, 'stories': 1680, 'business': 1665, 'war': 1663, 'more': 1651, 'it': 1636, 'manual': 1581, 'is': 1580, 'introduction': 1576, 'modern': 1555, 'second': 1555, 'management': 1553, 'one': 1534, 'america': 1533, 'all'

In [23]:
# group data by genre
# could probably be done MUCH nicer, but couldn't get for-loop to work...

grouped = train.groupby(train.genres)

Calendars = grouped.get_group("Calendars")
Comics = grouped.get_group("Comics & Graphic Novels")
Test = grouped.get_group("Test Preparation")
Mystery = grouped.get_group("Mystery, Thriller & Suspense")
SciFi = grouped.get_group("Science Fiction & Fantasy")
Romance = grouped.get_group("Romance")
Humor = grouped.get_group("Humor & Entertainment")
Literature = grouped.get_group("Literature & Fiction")
LGBTQ = grouped.get_group("Gay & Lesbian")
Engineering = grouped.get_group("Engineering & Transportation")
Food = grouped.get_group("Cookbooks, Food & Wine")
Crafts = grouped.get_group("Crafts, Hobbies & Home")
Arts = grouped.get_group("Arts & Photography")
Education = grouped.get_group("Education & Teaching")
Parenting = grouped.get_group("Parenting & Relationships")
SelfHelp = grouped.get_group("Self-Help")
Computers = grouped.get_group("Computers & Technology")
Medical = grouped.get_group("Medical Books")
Science = grouped.get_group("Science & Math")
Health = grouped.get_group("Health, Fitness & Dieting")
Business = grouped.get_group("Business & Money")
Law = grouped.get_group("Law")
Biographies = grouped.get_group("Biographies & Memoirs")
History = grouped.get_group("History")
Politics = grouped.get_group("Politics & Social Sciences")
Reference = grouped.get_group("Reference")
Bibles = grouped.get_group("Christian Books & Bibles")
Religion = grouped.get_group("Religion & Spirituality")
Sports = grouped.get_group("Sports & Outdoors")
Teen = grouped.get_group("Teen & Young Adult")
Childrens = grouped.get_group("Children's Books")
Travel = grouped.get_group("Travel")

GenreGroups = [Calendars['titles'], Comics['titles'], Test['titles'], Mystery['titles'], SciFi['titles'], 
               Romance['titles'], Humor['titles'], Literature['titles'], LGBTQ['titles'], Engineering['titles'], 
               Food['titles'], Crafts['titles'], Arts['titles'], Education['titles'], Parenting['titles'], 
               SelfHelp['titles'], Computers['titles'], Medical['titles'], Science['titles'], Health['titles'], 
               Business['titles'], Law['titles'], Biographies['titles'], History['titles'], Politics['titles'], 
               Reference['titles'], Bibles['titles'], Religion['titles'], Sports['titles'], Teen['titles'], 
               Childrens['titles'], Travel['titles']]

In [84]:
# token frequencies in each genre
genre_count = []
for g in GenreGroups:
    genre_title = Counter()
    for title in g:
        for i in title:
            genre_title[i] += 1
    genre_title = dict(genre_title)
    genre_count.append(genre_title)

In [146]:
# token frequencies in each genre
genre_count[0] # first index specifies genre, second index specifies token frequencies vs genre label

{'moms': 8,
 'family': 9,
 'wall': 815,
 'calendar': 1994,
 '2016': 1120,
 'doug': 1,
 'the': 370,
 'pug': 3,
 'moleskine': 30,
 'weekly': 57,
 'notebook': 17,
 '12m': 7,
 'large': 24,
 'black': 26,
 'soft': 8,
 'cover': 32,
 '5': 14,
 'x': 28,
 '825': 13,
 '365': 72,
 'cats': 36,
 'color': 13,
 'pageaday': 57,
 'sierra': 3,
 'club': 5,
 'engagement': 65,
 'wilderness': 9,
 'thomas': 11,
 'kinkade': 9,
 'disney': 20,
 'dreams': 3,
 'collection': 9,
 'ansel': 3,
 'adams': 3,
 'dilbert': 5,
 'daytoday': 86,
 'mary': 7,
 'engelbreit': 6,
 'deluxe': 33,
 'never': 3,
 'give': 1,
 'up': 8,
 'cat': 27,
 'gallery': 14,
 'llewellyns': 4,
 'witches': 3,
 'datebook': 3,
 'amy': 3,
 'knapp': 2,
 'big': 11,
 'grid': 3,
 'essential': 1,
 'organization': 1,
 'and': 137,
 'communication': 1,
 'tool': 1,
 'for': 67,
 'entire': 1,
 'outlander': 2,
 'audubon': 6,
 'nature': 15,
 'national': 21,
 'park': 3,
 'foundation': 1,
 'your': 13,
 'year': 41,
 'mindful': 3,
 'coloring': 13,
 'through': 29,
 'seaso

In [26]:
# number of words in a class
len(tok_freq_genres[0][1]) # first genre

2357

In [27]:
# number of total vocabulary (training set)
V = len(Counter(tok_freq))
print(V)

74688


In [28]:
# number of titles (in training set)
N_titles = len(train)
print(N_titles)

165976


In [29]:
# number of titles in each genre
N_genre = train['genres'].value_counts()
print(N_genre[:5])

Travel                       14693
Children's Books             10843
Medical Books                 9597
Health, Fitness & Dieting     9508
Business & Money              7987
Name: genres, dtype: int64


In [30]:
# priors of each genre (probability of title being in specific genre)
prob_title = []
for g in range(len(N_genre)):
    prob = N_genre[g] / N_titles
    prob_title.append(prob)

zipped_values = zip(df1.genre.unique(), prob_title)
prior = list(zipped_values)

print(prior[:5]) # test

[('Calendars', 0.08852484696582638), ('Comics & Graphic Novels', 0.06532872222489998), ('Test Preparation', 0.05782161276329108), ('Mystery, Thriller & Suspense', 0.05728539065889044), ('Science Fiction & Fantasy', 0.04812141514435822)]


In [139]:
# likelihoods of each word (probability of word being in specific genre)

# create empty dataframe
likelihood = pd.DataFrame(columns=df1.genre.unique(), index=dict.keys(tok_freq))

len_gc = -1
for i in genre_count: # loop through genres
    len_gc += 1 # create genre index
    for word in i: 
        p = (genre_count[len_gc][word] + 1) / (len(genre_count[len_gc]) + V) # probability
        likelihood.loc[word, likelihood.columns[len_gc]] = p # replace NaN in dataframe with p

In [191]:
# now fill probabilities of words that haven't appeared in genre

# loop through all words
# if NaN -> fill with p (0+1)/(word in genre + V)

#tok_freq

for r, index in likelihood.iterrows(): # loops through all rows (words)
  for c, v in likelihood.iteritems(): # loop through all columns -> returns tuple with genre + value
    for num_g in range(likelihood.shape[1]): # loop through number of columns so 0-32
      if np.isnan(likelihood.loc[r, c]) is True:
        p = 1 / (len(genre_count[num_g]) + V)
        likelihood.loc[r, c] = p
      else:
        pass

In [154]:
np.isnan(likelihood.loc["evidences", likelihood.columns[0]])

True

In [182]:
if np.isnan(likelihood.loc["moms", likelihood.columns[2]]) is True:
      p = 1 / (len(genre_count[2]) + V)
      likelihood.loc[r, likelihood.columns[2]] = p

In [185]:
p = 1 / (len(genre_count[2]) + V)
print(p)

1.2899896800825593e-05


In [186]:
likelihood.loc["moms", "Test Preparation"] = p

In [188]:
likelihood.columns[2]

'Test Preparation'

In [192]:
likelihood

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
moms,0.000116815,2.55118e-05,1.28999e-05,NaN,NaN,2.52283e-05,5.97843e-05,NaN,3.87222e-05,NaN,0.000120549,4.68664e-05,NaN,3.86284e-05,0.000304774,2.53694e-05,NaN,2.37164e-05,2.3733e-05,9.28548e-05,3.57007e-05,NaN,2.43424e-05,NaN,2.50335e-05,2.51209e-05,8.42693e-05,4.78435e-05,2.43075e-05,2.41217e-05,4.67935e-05,3.4099e-05
family,0.000129794,0.000191339,0.000219298,0.00010303,5.0491e-05,0.000428881,0.000370663,0.000361424,0.000335592,8.80791e-05,0.00232659,0.000421798,0.000253412,0.000180266,0.00176515,0.000202956,0.000122603,0.000545476,0.000166131,0.001358,0.000678313,0.00121721,0.000705931,0.000356366,0.000187751,0.000301451,0.00095104,0.000358826,0.000109384,0.000265338,0.000561522,0.00109117
wall,0.0105912,NaN,2.57998e-05,6.4394e-05,5.0491e-05,NaN,0.000239137,4.81899e-05,NaN,3.77482e-05,3.61646e-05,0.000234332,0.000229278,2.57523e-05,NaN,2.53694e-05,3.67809e-05,4.74327e-05,0.000118665,5.80343e-05,0.000714014,7.30327e-05,9.73698e-05,0.000166304,3.75502e-05,0.000163286,8.42693e-05,0.000167452,0.000157999,8.44258e-05,0.000187174,0.000431921
calendar,0.025894,NaN,NaN,NaN,NaN,NaN,0.000310878,4.81899e-05,NaN,0.000150993,3.61646e-05,0.000292915,0.000410286,NaN,6.34945e-05,7.61083e-05,6.13016e-05,NaN,0.000189864,6.96411e-05,5.95011e-05,NaN,NaN,NaN,2.50335e-05,5.02418e-05,7.22308e-05,0.000251178,0.000218768,2.41217e-05,5.84918e-05,0.000431921
2016,0.0145499,NaN,0.00136739,NaN,2.52455e-05,NaN,0.000298921,8.43323e-05,2.58148e-05,0.000452978,7.23292e-05,0.000480381,0.000458555,0.000193142,8.88923e-05,0.000114162,0.000514933,0.000557334,0.000142398,9.28548e-05,0.000321306,0.000121721,2.43424e-05,2.37578e-05,3.75502e-05,0.000376813,0.000180577,0.000346865,0.000255229,NaN,0.000116984,0.0013185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
evidences,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.27327e-05
4829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.27327e-05
flashmaps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.27327e-05
historymapped,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.27327e-05


In [147]:
# check whether value was actually replaced at specified index
likelihood.loc[["dog"]]

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
dog,0.000298527,6.37796e-05,NaN,3.86364e-05,8.83593e-05,3.78425e-05,0.000215223,0.000216854,NaN,NaN,4.82195e-05,0.00103106,9.65379e-05,NaN,NaN,5.07389e-05,2.45206e-05,0.000711491,9.49318e-05,0.000139282,3.57007e-05,4.86885e-05,0.000109541,3.56366e-05,3.75502e-05,7.53627e-05,NaN,2.39217e-05,0.00029169,4.82433e-05,0.000725299,0.000352357


In [ ]:
# test NB using test data

# predictions on word by word basis using words in titles of test data
# for each word in test title do: prior * likelihood^2

## BERT classifier

In [38]:
!pip install transformers

In [39]:
import matplotlib.pyplot as plt
import pandas as pd
import torch

# Preliminaries

from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [40]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [41]:
# Model parameter
MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

In [42]:
# Fields

label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)
text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False, include_lengths=False, batch_first=True,
                   fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)
fields = [('label', label_field), ('title', text_field), ('text', text_field), ('titletext', text_field)]

In [ ]:
# split data into test and train data -> use test and train variables

In [ ]:
# Iterators -> what is this?

In [43]:
# TabularDataset

train, valid, test = TabularDataset.splits(path=source_folder, train='train.csv', validation='valid.csv',
                                           test='test.csv', format='CSV', fields=fields, skip_header=True)

NameError: ignored

In [ ]:
# Iterators

train_iter = BucketIterator(train, batch_size=16, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=16, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)
test_iter = Iterator(test, batch_size=16, device=device, train=False, shuffle=False, sort=False)